In [1]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


In [2]:
from sklearn.base import BaseEstimator, ClassifierMixin

In [3]:
class StackedRegressor(BaseEstimator, ClassifierMixin):  
    def __init__(self, classifier=None, regressor=None):
        self.classifier = classifier
        self.regressor = regressor
        
    def fit(self, X, class_labels, regressor_labels):
        self.classifier.fit(X,class_labels)
        self.regressor.fit(X,regressor_labels)
        
    def predict(self, X, y=None):
        class_predict = self.classifier.predict(X)
        has_transaction = X[class_predict!=0]
        regressor_predict = self.regressor.predict(has_transaction)
        return regressor_predict

In [4]:
reg = LinearRegression()
classifier = LogisticRegression()

stacked = StackedRegressor(classifier=classifier, regressor=reg)

In [5]:
pd_data = pd.read_csv('../Data/numeric_csv_w_transaction.csv')

In [6]:
X = pd_data.iloc[:,0:31]
regressor_labels = pd_data.iloc[:,-1].fillna(0)


X_train, X_test, regressor_train, regressor_test = train_test_split(
    X, regressor_labels, test_size=0.33, random_state=42)

In [7]:
class_train = (regressor_train!=0).astype(int)
class_test = (regressor_test!=0).astype(int)


In [8]:
stacked.fit(X_train, class_train,regressor_train)

/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/bpc_msds2019/miniconda3/lib/python3.7/site-packages/sklearn/linear_model/base.py:485: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


In [9]:
pred = stacked.predict(X_test)

ValueError: Found array with 0 sample(s) (shape=(0, 31)) while a minimum of 1 is required.